# Request your knowledge data base

In [10]:
! uv pip install -qU wget

In [41]:
import json
import os
import re
import requests

import wget

In [42]:
base_url = "https://albert.api.etalab.gouv.fr/v1"
api_key = os.getenv("ALBERT_API_KEY")

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}

Let's begin by downloading document we're going to use as knowledge base. We're going to use a dataset from data.culture.gouv.fr :

In [43]:
doc_url = "https://data.culture.gouv.fr/api/explore/v2.1/catalog/datasets?limit=10&offset=0&timezone=UTC&include_links=false&include_app_metas=false"
file_path = "my_database.json"

if not os.path.exists(file_path):
    wget.download(doc_url, out=file_path)

File format must be JSON for our API endpoint, so no file's conversion required here. But we need to change file's structure to match API requirements. Moreover, file size must be under 20MB.

In [75]:
file = json.load(open(file_path))['results']

print("File content example :", file[0])
print("File size :", os.path.getsize(file_path) // 1024, "KB")

File content example : {'visibility': 'domain', 'dataset_id': 'base-des-reliures-numerisees-de-la-bibliotheque-nationale-de-france', 'dataset_uid': 'da_6v5zjp', 'has_records': False, 'features': [], 'attachments': [], 'alternative_exports': [], 'data_visible': True, 'fields': [], 'metas': {'default': {'title': 'Base des reliures numérisées de la Bibliothèque nationale de France', 'description': 'La BnF conserve l’une des plus importantes collections de reliures du monde dont il convenait d’améliorer le signalement pour en favoriser l’étude et la connaissance. Le site reliures.bnf.fr met ainsi en valeur des notices descriptives de reliures qui étaient jusqu’alors peu accessibles à un large public. Dans un premier temps, c’est la description des reliures françaises à décor couvrant la période du début du XVIe siècle au début du XIXe siècle qui a été privilégiée. Le site a été réalisé à partir de l’environnement de data.bnf.fr. Il permet d’agréger des données présentes dans le Catalogue g

Let's start by changing file structure. This one must a list of dictionnary with the following keys : 'title', 'text' and optionaly 'metadata'.

We're also going to clean text during this step to remove special characters and format it as expected.

In this example, we're going to import first 10  documents.

In [76]:
formated_file = list()
sample_size = 10
for document in file[:sample_size]:
    titre = document.get("metas").get("default").get("title", "")
    description = document.get("metas").get("default").get("description", "")
    intitule = document.get("dataset_id", "")

    text = re.sub(r"([.,;:!?])([^\s\d])", r"\1 \2", description)  # Add space after punctuation
    text = re.sub(r"[\xa0\u00a0\r]", " ", text)  # Remove special characters
    text = re.sub(r"&nbsp;", " ", text)
    text = re.sub(r"\,(?!\s)", ". ", text, count=1)  # Add a space after the first "," if not already followed by a space.

    formated_file.append(
        {
            "title": titre,
            "text": f"{titre} {text} {intitule}",
            "metadata": {"text": text,  "titre": titre, "intitule": intitule},
        }
    )

print(f"File content example : {formated_file[0]}")

File content example : {'title': 'Base des reliures numérisées de la Bibliothèque nationale de France', 'text': 'Base des reliures numérisées de la Bibliothèque nationale de France La BnF conserve l’une des plus importantes collections de reliures du monde dont il convenait d’améliorer le signalement pour en favoriser l’étude et la connaissance. Le site reliures. bnf. fr met ainsi en valeur des notices descriptives de reliures qui étaient jusqu’alors peu accessibles à un large public. Dans un premier temps, c’est la description des reliures françaises à décor couvrant la période du début du XVIe siècle au début du XIXe siècle qui a été privilégiée. Le site a été réalisé à partir de l’environnement de data. bnf. fr. Il permet d’agréger des données présentes dans le Catalogue général de la BnF, des représentations numérisées des reliures hébergées dans Gallica et en prévision les données du catalogue BnF Archives et Manuscrits. Ces données sont enrichies de descriptions, scientifiques et

Now we can upload files in Albert's vector database. 

First, let's create a collection named "tutorial". To do so, we need to find which model to use, so let's start by requesting '/v1/models' endpoint to get list of available models :

In [78]:
response = session.get(f"{base_url}/models")
response = response.json()
model = [model for model in response["data"] if model["type"] == "text-embeddings-inference"][0]["id"]

print(f"Embeddings model: {model}")

Embeddings model: embeddings-small


In [79]:
collection = "tutorial"

response = session.post(f"{base_url}/collections", json={"name": collection, "model": model})
response = response.json()
collection_id = response["id"]
print(f"Collection ID: {collection_id}")

Collection ID: 813


Now we use Albert API to upload files ine vector database :

In [66]:
batch = 2
for i in range(0, len(formated_file), batch):
    batch_file = formated_file[i : i + batch]

    batch_file_path = f"tmp_{i}.json"
    json.dump(batch_file, open(batch_file_path, "w"))
    assert os.path.getsize(batch_file_path) < 20 * 1024 * 1024

    files = {"file": (os.path.basename(batch_file_path), open(batch_file_path, "rb"), "application/json")}
    data = {"request": '{"collection": "%s"}' % collection_id}
    response = session.post(f"{base_url}/files", data=data, files=files)
    assert response.status_code == 201, "Erreur lors de l'importation du fichier"
    document_id = response.json()['id']
    os.remove(batch_file_path)

Let's check if uploaded file is available with GET `/v1/documents`.

In [67]:
response = session.get(f"{base_url}/documents/{document_id}")
response.json()

{'object': 'document',
 'id': 820297,
 'name': 'Données enquête utilisation des archives recherche universitaire.json',
 'collection_id': 812,
 'created_at': 1748876074,
 'chunks': 3}

OK, now you know how to create a knowledge base with Albert API. Before uploading your own files, let's remove those we uploaded for this tutorial :

In [80]:
response = session.delete(f"{base_url}/collections/{collection_id}")

In [81]:
response

<Response [204]>